In [103]:
# Import Python libraries
import pandas as pd
import numpy as np
from pandasgui import show
import pycountry

# Load CSV file of latest CF data
df_cf = pd.read_csv('cfdata-nov2020-3pm.csv')

# Calculate C and I revisions
df_cf = df_cf.assign(con_rev_2020 = lambda x: (100 * (1 + x.con2020nov / 100)) /  (100 * (1 + x.con2020jan / 100)) - 1,
                     con_rev_2021 = lambda x: ((100 * (1 + x.con2020nov / 100)) * (1 + x.con2021nov / 100)) /  ((100 * (1 + x.con2020jan / 100)) * (1 + x.con2021jan / 100)) - 1,
                     inv_rev_2020 = lambda x: (100 * (1 + x.inv2020nov / 100)) /  (100 * (1 + x.inv2020jan / 100)) - 1,
                     inv_rev_2021 = lambda x: ((100 * (1 + x.inv2020nov / 100)) * (1 + x.inv2021nov / 100)) /  ((100 * (1 + x.inv2020jan / 100)) * (1 + x.inv2021jan / 100)) - 1)

# Keep relevant columns only
df_cf = df_cf[['country', 'iso_code', 'con_rev_2020', 'con_rev_2021', 'inv_rev_2020', 'inv_rev_2021']]

# Extract COVID cases data from Our World in Data API
url_cc = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv'
df_cc = pd.read_csv(url_cc, parse_dates=['date'])
# Create a data frame for ln(last case per mil) for merging later
df_cc_short = pd.DataFrame(df_cc.groupby(['iso_code', 'location']).last()['total_cases_per_million']).reset_index()
# Generate ln(case per mil) and rename columns
df_cc_short = df_cc_short.assign(ln_case_per_mil = lambda x: np.log(x.total_cases_per_million))
df_cc_short.columns = ['iso_code', 'country', 'cases_per_mil', 'ln_case_per_mil']
# Merge df_cf and df_cc_short
df_cf = df_cf.set_index('iso_code').join(df_cc_short.set_index('iso_code'), rsuffix='_r').drop(columns='country_r').reset_index()
# Create a time series dataframe of total cases and cases per million
df_cc = df_cc[['iso_code', 'location', 'date', 'total_cases']]
# Rename columns for consistency
df_cc.columns = ['iso_code', 'country', 'date', 'total_cases']
# # Keep data points only from the 50th case onwards
# df_cc = df_cc[df_cc['total_cases'] >= 50]

# Extract stringency data from OxCGRT API
url_str = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
df_str = pd.read_csv(url_str, parse_dates=['Date'])
# Drop regional data and keep national level data only
df_str = df_str[df_str['RegionCode'].isnull()]
# Keep relevant columns only and rename them for merging
df_str = df_str[['CountryCode', 'CountryName', 'Date', 'StringencyIndex']]
df_str.columns = ['iso_code', 'country', 'date', 'stringency']
# Create a dataframe for second half of 2020 for merging
df_str_short = df_str[df_str['date'] >= '2020-06-01']
df_str_short = df_str_short.groupby(['iso_code', 'country']).mean().reset_index()
df_str_short.columns = ['iso_code', 'country', 'stringency_h2']
df_cf = df_cf.set_index('iso_code').join(df_str_short.set_index('iso_code'), rsuffix='_r').drop(columns=['country_r']).reset_index()

# Extract mobility data from Google API
url_mob = 'https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv'
df_mob = pd.read_csv(url_mob, parse_dates=['date'])
# Drop regional data and keep national level data only
df_mob = df_mob.loc[df_mob['sub_region_1'].isna()]
df_mob = df_mob.loc[df_mob['sub_region_2'].isna()]
df_mob = df_mob.loc[df_mob['metro_area'].isna()]
# Use workplace mobility, rename columns, and get iso_code
df_mob = df_mob[['country_region', 'date', 'workplaces_percent_change_from_baseline']]
df_mob.columns = ['country', 'date', 'mobility']
input_countries = df_mob['country'].values
countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_3
codes = [countries.get(country, 'Unknown code') for country in input_countries]
df_mob['iso_code'] = codes
df_mob = df_mob[['iso_code', 'country', 'date', 'mobility']]
# Create a dataframe for second half of 2020 for merging later
df_mob_short = df_mob[df_mob['date'] >= '2020-06-01']
df_mob_short = df_mob_short.groupby(by='country').mean().reset_index()
df_mob_short.columns = ['country', 'mobility_h2']
df_cf = df_cf.set_index('country').join(df_mob_short.set_index('country')).reset_index()


# Merge dataframes
df_cc_str = df_cc.set_index(['iso_code', 'date']).join(df_str.set_index(['iso_code', 'date']), rsuffix = '_r').drop(columns=['country_r']).reset_index()
df_cc_str_mob = df_cc_str.dropna(subset=['iso_code']).set_index(['iso_code', 'date']).join(df_mob.set_index(['iso_code', 'date']), rsuffix = '_r').drop(columns=['country_r']).reset_index()
df_cc_str_mob = df_cc_str_mob[df_cc_str_mob['total_cases'] > 50]
df_cc_str_mob = df_cc_str_mob.groupby('iso_code').mean()[['mobility', 'stringency']].reset_index()
df_cc_str_mob = df_cc_str_mob.groupby('iso_code').mean()[['mobility', 'stringency']].reset_index()
df_all = df_cf.set_index('iso_code').join(df_cc_str_mob.set_index('iso_code')).reset_index()
df_all = df_all[['country', 'iso_code', 'mobility', 'mobility_h2', 'stringency', 'stringency_h2', 
                 'cases_per_mil', 'ln_case_per_mil', 'con_rev_2020', 'con_rev_2021', 'inv_rev_2020',
                 'inv_rev_2021']]

# Calculate average mobility for others (<50 cases and mismatched iso_codes)
df_all.loc[df_all['iso_code']=='FJI', 'mobility'] = df_mob.loc[df_mob['iso_code']=='FJI'][df_mob.loc[df_mob['iso_code']=='FJI']['date']>='2020-04-01'].mobility.mean()
df_all.loc[df_all['iso_code']=='HKG', 'mobility'] = df_mob.loc[df_mob['iso_code']=='HKG'][df_mob.loc[df_mob['iso_code']=='HKG']['date']>='2020-02-13'].mobility.mean()
df_all.loc[df_all['iso_code']=='LAO', 'mobility'] = df_mob.loc[df_mob['country']=='Laos'][df_mob.loc[df_mob['country']=='Laos']['date']>='2020-04-01'].mobility.mean()
df_all.loc[df_all['iso_code']=='MMR', 'mobility'] = df_mob.loc[df_mob['country']=='Myanmar (Burma)'][df_mob.loc[df_mob['country']=='Myanmar (Burma)']['date']>='2020-04-13'].mobility.mean()
df_all.loc[df_all['iso_code']=='RUS', 'mobility'] = df_mob.loc[df_mob['country']=='Russia'][df_mob.loc[df_mob['country']=='Russia']['date']>='2020-04-01'].mobility.mean()
df_all.loc[df_all['iso_code']=='KOR', 'mobility'] = df_mob.loc[df_mob['country']=='South Korea'][df_mob.loc[df_mob['country']=='South Korea']['date']>='2020-02-20'].mobility.mean()
df_all.loc[df_all['iso_code']=='TWN', 'mobility'] = df_mob.loc[df_mob['country']=='Taiwan'][df_mob.loc[df_mob['country']=='Taiwan']['date']>='2020-04-01'].mobility.mean()
df_all.loc[df_all['iso_code']=='VNM', 'mobility'] = df_mob.loc[df_mob['country']=='Vietnam'][df_mob.loc[df_mob['country']=='Vietnam']['date']>='2020-04-01'].mobility.mean()

# Calculate average stringency for others (<50 cases and mismatched iso_codes)
df_all.loc[df_all['iso_code']=='FJI', 'stringency'] = df_str.loc[df_str['iso_code']=='FJI'][df_str.loc[df_str['iso_code']=='FJI']['date']>='2020-04-01'].stringency.mean()
df_all.loc[df_all['iso_code']=='HKG', 'stringency'] = df_str.loc[df_str['iso_code']=='HKG'][df_str.loc[df_str['iso_code']=='HKG']['date']>='2020-02-13'].stringency.mean()
df_all.loc[df_all['iso_code']=='LAO', 'stringency'] = df_str.loc[df_str['iso_code']=='LAO'][df_str.loc[df_str['iso_code']=='LAO']['date']>='2020-04-01'].stringency.mean()
df_all.loc[df_all['iso_code']=='SLB', 'stringency'] = df_str.loc[df_str['iso_code']=='SLB'][df_str.loc[df_str['iso_code']=='SLB']['date']>='2020-04-01'].stringency.mean()
df_all.loc[df_all['iso_code']=='TLS', 'stringency'] = df_str.loc[df_str['iso_code']=='TLS'][df_str.loc[df_str['iso_code']=='TLS']['date']>='2020-04-01'].stringency.mean()
df_all.loc[df_all['iso_code']=='TKM', 'stringency'] = df_str.loc[df_str['iso_code']=='TKM'][df_str.loc[df_str['iso_code']=='TKM']['date']>='2020-04-01'].stringency.mean()
df_all.loc[df_all['iso_code']=='VUT', 'stringency'] = df_str.loc[df_str['iso_code']=='VUT'][df_str.loc[df_str['iso_code']=='VUT']['date']>='2020-04-01'].stringency.mean()

df_all.to_excel('cfdata-november.xlsx')

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning:

Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning:

Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.

